In [2]:
import numpy as np
import pandas as pd
import yaml

In [1]:
model_path = '/scratch/c.c21013066/data/ukbiobank/analyses/acc_models'
data_path = '/scratch/c.c21013066/data/ukbiobank'

In [3]:
def read_traits_file(input_path: str):
    with open(input_path, 'r') as f:
        traits_data = yaml.load(f, Loader=yaml.BaseLoader)
    return traits_data

In [4]:
dfs = pd.read_csv(f'{data_path}/phenotypes/accelerometer/matched_all_HCnoOsteo_genebloodrisk_prodromalsigns.csv',index_col=0)
subset = dfs[dfs['diagnosis']=='ParkinsonDisease']
covs = ['accelerometry_age','male']
classes = ['sleep','light','sedentary','MVPA','imputed']
allfeatures = np.hstack([dfs.columns[79:117],
                  [f'mean_{cl}_hours_per24h' for cl in classes],
                  [f'std_{cl}_hours_per24h' for cl in classes],
                  [f'mean_movement_during_{cl}' for cl in classes],
                  [f'std_movement_during_{cl}' for cl in classes],
                  [f'mean_max_{cl}_hours_consecutive_per24h' for cl in classes],
                  [f'max_{cl}_hours_consecutive' for cl in classes],
                  [f'mean_N_{cl}_intervals_per24h' for cl in classes],
                  [f'mean_N_{cl}_intervals_07-23' for cl in classes],
                  [f'mean_N_{cl}_intervals_23-07' for cl in classes],covs])
scale_allfeatures = allfeatures[:-1]
# get subset of features
# drop features with too many nan
drop = allfeatures[subset[allfeatures].isna().sum() > 0]
subset = subset.drop(columns=drop)
allfeatures = list(set(allfeatures).difference(set(drop)))
scale_allfeatures = list(set(scale_allfeatures).difference(set(drop)))

np.save(f'{model_path}/allfeatures',allfeatures)

In [8]:
# population cohort
# get data set of PD, prod PD and matched controls
dfs = pd.read_csv(f'{data_path}/phenotypes/accelerometer/unmatched_all_HCnoOsteo_genebloodrisk_prodromalsigns.csv',index_col=0)
subset = dfs[dfs['diagnosis']=='ParkinsonDisease']

targets = ['diagnosis','Status','eid']
# train for robustness
# include all other diseases, but only once
unique = dfs[dfs['diagnosis']!="ParkinsonDisease"].dropna(subset=allfeatures,how='any')
# exclude depression cases
unique = dfs[dfs['diagnosis']!="Depression"]
unique = unique[~unique.index.duplicated(keep='last')]
unique = pd.concat([unique,subset])
unique = unique[~unique.index.duplicated(keep='last')]
unique['diag_PDHC'] = (unique['diagnosis'] == 'ParkinsonDisease').astype(int) * unique['Status'].replace(['Prodromal','Diseased','Healthy'],[0,1,0])
unique['diag_ProdHC'] = (unique['diagnosis'] == 'ParkinsonDisease').astype(int) * unique['Status'].replace(['Prodromal','Diseased','Healthy'],[1,0,0])
unique['diag_PDProdPopulation'] = (unique['diagnosis'] == 'ParkinsonDisease').astype(int) * unique['Status'].replace(['Prodromal','Diseased','Healthy'],[1,1,0])
unique['diag_PDPopulation'] = (unique['diagnosis'] == 'ParkinsonDisease').astype(int) * unique['Status'].replace(['Prodromal','Diseased','Healthy'],[0,1,0])
unique['diag_ProdPopulation'] = (unique['diagnosis'] == 'ParkinsonDisease').astype(int) * unique['Status'].replace(['Prodromal','Diseased','Healthy'],[1,0,0])
unique['diag_ProdPopulationNoPD'] = (unique['diagnosis'] == 'ParkinsonDisease').astype(int) * unique['Status'].replace(['Prodromal','Diseased','Healthy'],[1,0,0])
unique.loc[np.logical_and(unique['diagnosis']=='ParkinsonDisease',unique['Status']=='Diseased'),'diag_ProdPopulationNoPD'] = np.nan
unique['diag_PDPopulationNoProd'] = (unique['diagnosis'] == 'ParkinsonDisease').astype(int) * unique['Status'].replace(['Prodromal','Diseased','Healthy'],[0,1,0])
unique.loc[np.logical_and(unique['diagnosis']=='ParkinsonDisease',unique['Status']=='Prodromal'),'diag_PDPopulationNoProd'] = np.nan

# get data of PRS
traits = read_traits_file('../../resources/genetics/traits.yaml')
traits = pd.DataFrame(traits)
score1 = pd.read_csv(f'{data_path}/ukb52375.csv').set_index('eid')
trait='26260-0.0'
score_best = score1[trait]
score1.columns = score1.columns.str.replace('-0.0','')
PRSs = score1[traits.columns]
PRSs.columns = traits.loc['full_name',PRSs.columns]
genetics = PRSs.columns
genetics_scale = genetics

# merge data
merged = pd.merge(unique,score_best,right_index=True,left_index=True,how='left').rename(columns={trait:'PRS'})
merged = pd.merge(merged,PRSs,right_index=True,left_index=True,how='left',suffixes=['_drop',''])
merged = merged.drop(columns=merged.filter(regex="_drop").columns)

# add BMI etc features
lifestyle = pd.read_csv(f'{data_path}/phenotypes/accelerometer/unmatched_all_HCnoOsteo_riskblood.csv',index_col=0)
lifestylePD = lifestyle[lifestyle['diagnosis']=='ParkinsonDisease']
lifestyle = lifestyle[lifestyle['diagnosis']!="ParkinsonDisease"].dropna(subset=allfeatures,how='any')
lifestyle = lifestyle[~lifestyle.index.duplicated(keep='last')]
lifestyle = pd.concat([lifestyle,lifestylePD])
lifestyle = lifestyle[~lifestyle.index.duplicated(keep='last')]
life_cols = lifestyle.columns[190:]#lifestyle.columns[141:-1]
lifestyle = lifestyle[life_cols]
#drop = ['AllCauseDementia','visit_age','time_to_diagnosis','AllCauseDementia_age','male','TownsendDeprivationIndex']
#lifestyle = lifestyle.drop(columns=drop)
life_cols = np.hstack([lifestyle.columns[:17],'TownsendDeprivationIndex'])
life_scale = life_cols[11:]
blood_cols = lifestyle.columns[17:]
blood_scale = blood_cols
family_cols = ['family_Stroke', 'family_Diabetes', 'family_Severedepression',
       'family_Alzheimersdiseasedementia', 'family_Parkinsonsdisease']
family_scale = family_cols
life_nofam_cols = np.hstack([life_cols[:6],life_cols[11:]])
life_nofam_scale = life_nofam_cols

prodromal = ['UrinaryIncontinence','Constipation','ErectileDysfunction','Anxiety','RBD','Hyposmia','OrthostaticHypotension',
                'Depression']
prod_col = [f'{p}_beforePD' for p in prodromal]
prod_acc = [f'{p}_beforeacc' for p in prodromal]

merged = pd.merge(merged,lifestyle,right_index=True,left_index=True,how='left',suffixes=['_drop',''])
merged = merged.drop(columns=merged.filter(regex='_drop').columns)
features_all = np.hstack([allfeatures,blood_cols,life_cols,genetics,prod_acc,prod_col])
nona = merged.dropna(subset=np.hstack(['diag_PDProdPopulation',features_all]))

/tmp/ipykernel_248561/450464092.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  score1.columns = score1.columns.str.replace('-0.0','')


In [65]:
# OLD: for matched HC population cohort
# drop HC match for each prodromal/diseased without complete info
eids = pd.DataFrame(index=merged[np.logical_and(merged[features_all].isna().sum(axis=1)>1,merged['Status'].isin(['Prodromal','Diseased']))].index,
                     columns=['control_match'])
match_cols = ['accelerometry_age','male']
for key,row in merged[merged[features_all].isna().sum(axis=1)>1].iterrows():
    control = merged[np.logical_and(merged['diagnosis']==row['diagnosis'],merged['Status']=='Healthy')]
    if row['Status']=='Prodromal':
        control = control[control['Group']=='Healthy_Prodromal']
    elif row['Status']=='Diseased':
        control = control[control['Group']=='Healthy_Diseased']
    match = control[(control[match_cols[0]].round()==np.round(row[match_cols[0]])) & (control[match_cols[1]]==row[match_cols[1]])].sample(n=1)
    eids.loc[key,'control_match'] = match.index.values[0]
    control = control[~control.index.isin(eids['control_match'])]
    merged = merged.drop(index=match.index)
nona = merged.dropna(subset=np.hstack(['diag_PDProdPopulation',features_all]))

ValueError: a must be greater than 0 unless no samples are taken

In [9]:
# get population cohort which uses all available HC
models = ['diag_ProdPopulationNoPD','diag_PDPopulationNoProd','diag_PDProdPopulation']
hc = pd.read_csv(f'{data_path}/phenotypes/accelerometer/allHCnoOsteo_prodromalsigns.csv',
                      index_col=0)
for name in models:
    hc[name] = 0
hc = pd.merge(hc,score_best,right_index=True,left_index=True,how='left').rename(columns={trait:'PRS'})
hc = pd.merge(hc,PRSs,right_index=True,left_index=True,how='left',suffixes=['_drop',''])
hc = hc.drop(columns=hc.filter(regex="_drop").columns)
mergedHC = pd.concat([merged[np.hstack([models,features_all,'Status','diagnosis','date_accelerometry'])],hc[np.hstack([models,features_all,'Status','diagnosis','date_accelerometry'])]])

nonaHC = mergedHC.dropna(subset=features_all,how='any',axis='rows')
nonaHC = nonaHC[~nonaHC.index.duplicated(keep='first')]

In [ ]:
#nona.to_csv('/scratch/c.c21013066/data/ukbiobank/merged_data/populationNoOsteoMatchedHC.csv')
nonaHC.to_csv('/scratch/c.c21013066/data/ukbiobank/merged_data/populationNoOsteoAllHC.csv')

In [4]:
#nona = pd.read_csv('/scratch/c.c21013066/data/ukbiobank/merged_data/populationNoOsteoMatchedHC.csv').set_index('eid')
nonaHC = pd.read_csv('/scratch/c.c21013066/data/ukbiobank/merged_data/populationNoOsteoAllHC.csv').set_index("eid")

stats = nonaHC.groupby('diag_PDPopulationNoProd')[features_all].agg(['mean','std','size'])
stats = stats.rename(index={0:'population',1:'diagnosed'})
stats2 = nonaHC.groupby('diag_ProdPopulationNoPD')[features_all].agg(['mean','std','size'])
stats2 = stats2.rename(index={0:'drop',1:'prodromal'})

NameError: name 'features_all' is not defined

In [5]:
# get data set of PD, prod PD and matched controls
dfs = pd.read_csv(f'{data_path}/phenotypes/accelerometer/matched_all_HCnoOsteo_genebloodrisk_prodromalsigns.csv',index_col=0)
name = 'ParkinsonDisease'
subset = dfs[dfs['diagnosis']==name]

targets = ['diagnosis','Status','eid']
subset['diag_PDHC'] = subset['Status_group'].replace(['Prodromal','Diseased'],[np.nan,1]) * subset['Status'].replace(['Prodromal','Diseased','Healthy'],[np.nan,1,0])
subset['diag_ProdHC'] = subset['Status_group'].replace(['Prodromal','Diseased'],[1,np.nan]) * subset['Status'].replace(['Prodromal','Diseased','Healthy'],[1,np.nan,0])
subset['diag_PDProdHC'] = subset['Status'].replace(['Prodromal','Diseased','Healthy'],[1,1,0])

# get data of PRS
traits = read_traits_file('../../resources/genetics/traits.yaml')
traits = pd.DataFrame(traits)
score1 = pd.read_csv(f'{data_path}/ukb52375.csv').set_index('eid')
trait='26260-0.0'
score_best = score1[trait]
score1.columns = score1.columns.str.replace('-0.0','')
PRSs = score1[traits.columns]
PRSs.columns = traits.loc['full_name',PRSs.columns]
genetics = PRSs.columns
genetics_scale = genetics

# merge data
merged = pd.merge(subset,score_best,right_index=True,left_index=True,how='left').rename(columns={trait:'PRS'})
merged = pd.merge(merged,PRSs,right_index=True,left_index=True,how='left',suffixes=['_drop',''])
merged = merged.drop(columns=merged.filter(regex="_drop").columns)

# add BMI etc features
lifestyle = pd.read_csv(f'{data_path}/phenotypes/accelerometer/matched_all_HCnoOsteo_riskblood.csv',index_col=0)
lifestyle = lifestyle[lifestyle['diagnosis']==name]
life_cols = lifestyle.columns[193:]
lifestyle = lifestyle[life_cols]
#drop = ['AllCauseDementia','visit_age','time_to_diagnosis','AllCauseDementia_age','male','TownsendDeprivationIndex']
#lifestyle = lifestyle.drop(columns=drop)
life_cols = np.hstack([lifestyle.columns[:17],'TownsendDeprivationIndex'])
life_scale = life_cols[11:]
blood_cols = lifestyle.columns[17:]
blood_scale = blood_cols
family_cols = ['family_Stroke', 'family_Diabetes', 'family_Severedepression',
       'family_Alzheimersdiseasedementia', 'family_Parkinsonsdisease']
family_scale = family_cols
life_nofam_cols = np.hstack([life_cols[:6],life_cols[11:]])
life_nofam_scale = life_nofam_cols

prodromal = ['UrinaryIncontinence','Constipation','ErectileDysfunction','Anxiety','RBD','Hyposmia','OrthostaticHypotension',
                'Depression']
prod_col = [f'{p}_beforePD' for p in prodromal]
prod_acc = [f'{p}_beforeacc' for p in prodromal]

merged = pd.merge(merged,lifestyle,right_index=True,left_index=True,how='left',suffixes=['_drop',''])
merged = merged.drop(columns=merged.filter(regex='_drop').columns)
features_all = np.hstack([allfeatures,blood_cols,life_cols,genetics,prod_acc,prod_col])

/tmp/ipykernel_248561/2280073070.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['diag_PDHC'] = subset['Status_group'].replace(['Prodromal','Diseased'],[np.nan,1]) * subset['Status'].replace(['Prodromal','Diseased','Healthy'],[np.nan,1,0])
/tmp/ipykernel_248561/2280073070.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['diag_ProdHC'] = subset['Status_group'].replace(['Prodromal','Diseased'],[1,np.nan]) * subset['Status'].replace(['Prodromal','Diseased','Healthy'],[1,np.nan,0])
/tmp/ipy

In [232]:
# HC MATCHED to PD cases
# drop HC match for each prodromal/diseased without complete info
eids = pd.DataFrame(index=merged[np.logical_and(merged[features_all].isna().sum(axis=1)>1,merged['Status'].isin(['Prodromal','Diseased']))].index,
                     columns=['control_match'])
match_cols = ['accelerometry_age','male']
merged_clean = merged.copy(deep=True)
for key,row in merged_clean[merged_clean[features_all].isna().sum(axis=1)>1].iterrows():
    control = merged_clean[np.logical_and(merged_clean['diagnosis']==row['diagnosis'],merged_clean['Status']=='Healthy')]
    if row['Status']=='Prodromal':
        control = control[control['Group']=='Healthy_Prodromal']
    elif row['Status']=='Diseased':
        control = control[control['Group']=='Healthy_Diseased']
    match = control[(control[match_cols[0]].round()==np.round(row[match_cols[0]])) & (control[match_cols[1]]==row[match_cols[1]])].sample(n=1)
    eids.loc[key,'control_match'] = match.index.values[0]
    control = control[~control.index.isin(eids['control_match'])]
    merged_clean = merged_clean.drop(index=match.index)
# drop all with incomplete data
nonamatched = merged_clean.dropna(subset=np.hstack(['diag_PDProdHC',features_all]))

In [6]:
# get population cohort which uses all available HC
nona = merged.dropna(subset=np.hstack(['diag_PDProdHC',features_all]))
models = ['diag_ProdHC','diag_PDHC','diag_PDProdHC']
hc = pd.read_csv(f'{data_path}/phenotypes/accelerometer/allHCnoOsteo_prodromalsigns.csv',
                      index_col=0)
for name in models:
    hc[name] = 0
hc = pd.merge(hc,score_best,right_index=True,left_index=True,how='left').rename(columns={trait:'PRS'})
hc = pd.merge(hc,PRSs,right_index=True,left_index=True,how='left',suffixes=['_drop',''])
hc = hc.drop(columns=hc.filter(regex="_drop").columns)
mergedHC = pd.concat([merged[np.hstack([models,features_all,'Status','diagnosis','date_accelerometry'])],hc[np.hstack([models,features_all,'Status','diagnosis','date_accelerometry'])]])

nonaHC = mergedHC.dropna(subset=features_all,how='any',axis='rows')
nonaHC = nonaHC[~nonaHC.index.duplicated(keep='last')]

In [7]:
#nonamatched.to_csv('/scratch/c.c21013066/data/ukbiobank/merged_data/unaffectedNoOsteoMatchedHC.csv')
nonaHC.to_csv('/scratch/c.c21013066/data/ukbiobank/merged_data/unaffectedNoOsteoAllHC.csv')

In [235]:
nona = pd.read_csv('/scratch/c.c21013066/data/ukbiobank/merged_data/unaffectedNoOsteoMatchedHC.csv').set_index('eid')
nonaHC = pd.read_csv('/scratch/c.c21013066/data/ukbiobank/merged_data/unaffectedNoOsteoAllHC.csv').set_index("eid")

stats3 = nona.groupby('diag_PDHC')[features_all].agg(['mean','std','size'])
stats3 = stats3.rename(index={0:'unaffected control matches for diagnosed',1:'diagnosed'})
stats4 = nona.groupby('diag_ProdHC')[features_all].agg(['mean','std','size'])
stats4 = stats4.rename(index={0:'unaffected control matches for prodromal',1:'prodromal'})
stats5 = nonaHC.groupby('diag_PDHC')[features_all].agg(['mean','std','size'])
stats5 = stats5.rename(index={0:'unaffected controls',1:'diagnosed'})

In [236]:
statss = pd.concat([stats,stats2,stats3,stats4,stats5])

In [238]:
statss.drop_duplicates(keep='first').T.to_csv('/scratch/c.c21013066/data/ukbiobank/phenotypes/data_sumstat_HCnoOsteo.csv')

In [237]:
statss.drop_duplicates(keep='first').T

population   diagnosed   prodromal  \
std_sedentary_hours_per24h      mean      1.507658    1.481726    1.390165   
                                std       0.567849    0.532759    0.477680   
                                size  33009.000000  153.000000  113.000000   
mean_N_MVPA_intervals_23-07     mean      0.094652    0.058824    0.061947   
                                std       0.345245    0.185819    0.297305   
...                                            ...         ...         ...   
OrthostaticHypotension_beforePD std       0.081367    0.080845    0.242133   
                                size  33009.000000  153.000000  113.000000   
Depression_beforePD             mean      0.041716    0.071895    0.061947   
                                std       0.199942    0.259163    0.242133   
                                size  33009.000000  153.000000  113.000000   

                                      unaffected control matches for diagnosed  \
std_sedentary_hours_per24h      mean                                  1.459113   
                                std                                   0.561370   
                                size                                153.000000   
mean_N_MVPA_intervals_23-07     mean                                  0.061983   
                                std                                   0.166584   
...                                                                        ...   
OrthostaticHypotension_beforePD std                                   0.080845   
                                size                                153.000000   
Depression_beforePD             mean                                  0.000000   
                                std                                   0.000000   
                                size                                153.000000   

                                      unaffected control matches for prodromal  \
std_sedentary_hours_per24h      mean                                  1.428016   
                                std                                   0.495947   
                                size                                113.000000   
mean_N_MVPA_intervals_23-07     mean                                  0.065192   
                                std                                   0.196163   
...                                                                        ...   
OrthostaticHypotension_beforePD std                                   0.132443   
                                size                                113.000000   
Depression_beforePD             mean                                  0.000000   
                                std                                   0.000000   
                                size                                113.000000   

                                      unaffected controls  
std_sedentary_hours_per24h      mean             1.530049  
                                std              0.576404  
                                size         24987.000000  
mean_N_MVPA_intervals_23-07     mean             0.103926  
                                std              0.364953  
...                                                   ...  
OrthostaticHypotension_beforePD std              0.059909  
                                size         24987.000000  
Depression_beforePD             mean             0.000000  
                                std              0.000000  
                                size         24987.000000  

[525 rows x 6 columns]